In [1]:
from pytz import timezone
import matplotlib.pyplot as plt
import pandas as pd
import time
import datetime as dt
from datetime import datetime
import MetaTrader5 as mt5
import pandas as pd
import pytz
import plotly.graph_objects as go


utc_tz = timezone('UTC')
pd.set_option('display.max_columns', 500) # number of columns to be displayed
pd.set_option('display.width', 1500)      # max table width to display

In [2]:
# This will need to be updated as it should come directly from Currency Strength Script
MS = 'Bearish'
Symbol = 'EURUSD'

In [3]:
if not mt5.initialize():
        print("initialize() failed, error code =",mt5.last_error())
        quit()
mt5.login()
timezone = pytz.timezone("UTC")
now = datetime.now(timezone)
start = datetime.now(timezone) - dt.timedelta(days=5)
utc_from = datetime(start.year, start.month, start.day)
utc_to = datetime(now.year, now.month, now.day, now.hour, now.minute, now.second)
rates = mt5.copy_rates_range(Symbol, mt5.TIMEFRAME_M15, utc_from, utc_to)
htf = pd.DataFrame(rates)
htf['time']=pd.to_datetime(htf['time'], unit='s')
start=time.time()

In [4]:
htf

,time,open,high,low,close,tick_volume,spread,real_volume
0,2022-08-15 04:00:00,1.02663,1.02684,1.02609,1.02610,916,0,0
1,2022-08-15 04:15:00,1.02613,1.02645,1.02550,1.02551,902,0,0
2,2022-08-15 04:30:00,1.02552,1.02600,1.02532,1.02576,857,0,0
3,2022-08-15 04:45:00,1.02578,1.02584,1.02553,1.02573,574,0,0
4,2022-08-15 05:00:00,1.02573,1.02574,1.02476,1.02488,927,0,0
...,...,...,...,...,...,...,...,...
459,2022-08-19 22:45:00,1.00382,1.00408,1.00358,1.00402,635,0,0
460,2022-08-19 23:00:00,1.00402,1.00431,1.00392,1.00408,415,0,0
461,2022-08-19 23:15:00,1.00408,1.00427,1.00399,1.00415,189,0,0
462,2022-08-19 23:30:00,1.00415,1.00417,1.00398,1.00407,139,2,0


In [5]:
for i in range(2,htf.shape[0]):
    current = htf.iloc[i,:]
    prev = htf.iloc[i-1,:]
    prev_2 = htf.iloc[i-2,:]
    realbody = abs(current['open'] - current['close'])
    candle_range = current['high'] - current['low']
    idx = htf.index[i]
    htf.loc[idx,'Bullish swing'] = current['low'] > prev['low'] and prev['low'] < prev_2['low']
    htf.loc[idx,'Bearish swing'] = current['high'] < prev['high'] and prev['high'] > prev_2['high']
    #Finding the Bullish Market Shift
    if htf.loc[idx,'Bullish swing'] == True:
        htf.loc[idx,'Bull1'] = htf['low'].loc[htf.index[i-1]]
        Bull1 = htf.loc[htf.index[i-1]]
    if htf.loc[idx,'Bearish swing'] == True and Bull1['time'] < htf.loc[idx,'time']:
        htf.loc[idx,'Bull2'] = htf['high'].loc[htf.index[i-1]]
        Bull2 = htf.loc[htf.index[i-1]]
    if htf.loc[idx,'Bullish swing'] == True and Bull1['time'] < htf.loc[idx,'time']:
        htf.loc[idx,'Bull3'] = htf['low'].loc[htf.index[i-1]]
        Bull3 = htf.loc[htf.index[i-1]]
    #Finding the Bearish Market Shift
    if htf.loc[idx,'Bearish swing'] == True:
        htf.loc[idx,'Bear1'] = htf['high'].loc[htf.index[i-1]]
        Bear1 = htf.loc[htf.index[i-1]]
    if htf.loc[idx,'Bearish swing'] == True and Bear1['time'] < htf.loc[idx,'time']:
        htf.loc[idx,'Bear2'] = htf['low'].loc[htf.index[i-1]]
        Bear2 = htf.loc[htf.index[i-1]]
    if htf.loc[idx,'Bearish swing'] == True and Bear1['time'] < htf.loc[idx,'time']:
        htf.loc[idx,'Bear3'] = htf['high'].loc[htf.index[i-1]]
        Bear3 = htf.loc[htf.index[i-1]]

In [6]:
htf.tail(15)

,time,open,high,low,close,tick_volume,spread,real_volume,Bullish swing,Bearish swing,Bull1,Bull3,Bull2,Bear1,Bear2,Bear3
449,2022-08-19 20:15:00,1.00433,1.00454,1.00405,1.00415,562,0,0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
450,2022-08-19 20:30:00,1.00414,1.00419,1.00369,1.00408,543,0,0,False,True,NaN,NaN,1.00454,1.00454,1.00405,1.00454
451,2022-08-19 20:45:00,1.00408,1.00449,1.00405,1.00434,609,0,0,True,False,1.00369,1.00369,NaN,NaN,NaN,NaN
452,2022-08-19 21:00:00,1.00434,1.00434,1.00353,1.00356,619,0,0,False,True,NaN,NaN,1.00449,1.00449,1.00405,1.00449
453,2022-08-19 21:15:00,1.00354,1.00440,1.00353,1.00418,601,0,0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
454,2022-08-19 21:30:00,1.00419,1.00460,1.00384,1.00396,559,0,0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
455,2022-08-19 21:45:00,1.00396,1.00397,1.00322,1.00330,608,0,0,False,True,NaN,NaN,1.00460,1.00460,1.00384,1.00460
456,2022-08-19 22:00:00,1.00333,1.00362,1.00321,1.00356,694,0,0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
457,2022-08-19 22:15:00,1.00356,1.00375,1.00346,1.00359,499,0,0,True,False,1.00321,1.00321,NaN,NaN,NaN,NaN
458,2022-08-19 22:30:00,1.00359,1.00390,1.00359,1.00384,548,0,0,False,False,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
if MS == 'Bullish':
    if htf.loc[idx,'Bullish swing'] == True:
        bull_low1 = Bull1['low']
    if Bull1['time'] < Bull2['time']:
        bull_high = Bull2['high']
        bulldis_high = bull_high
    if Bull2['time'] < Bull3['time'] == True:
        bull_low2 = Bull3['low']
        bulldis_low = bull_low2
    if Bull1['low'] > Bull3['low']:
        start_time = Bull1['time']
        end_time = Bull3['time']
        print ('Start time is:',start_time)
        print ('End time is:',end_time)
        print ('Price Entry Zone is from', bulldis_low, 'to', bulldis_high)
    else:
        print ("No Pattern Found!")
else:
    if htf.loc[325,'Bearish swing'] == True:
        Bear_low1 = Bear1['high']
    if Bear1['time'] < Bear2['time']:
        Bear_high = Bear2['low']
        beardis_high = Bear_high
    if Bear2['time'] < Bear3['time'] == True:
        Bear_low2 = Bear3['high']
        beardis_low = Bear_low2
    if Bear1['low'] > Bear3['low']:
        start_time = Bear1['high']
        end_time = Bear3['time']
        print ('Start time is:',start_time)
        print ('End time is:',end_time)
        print ('Price Entry Zone is from', beardis_low, 'to', beardis_high)
    else:
        print ("No Pattern Found!")

No Pattern Found!


In [8]:
end=time.time()
time_taken=end-start
print(time_taken)

0.6749625205993652


In [9]:
time_format = time.strftime("%H:%M:%S", time.gmtime(time_taken*30))
time_format

'00:00:20'